In [2]:
from Executor import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("segmented").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/segmented/").\
    build()

Pretrained Vgg16 model loaded.
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
initialized training data from: ../data/segmented/train
initialized validation data from: ../data/segmented/valid
found number of softmax classes: 3


In [4]:
executor.precompute_conv_model_outputs().and_().save_precomputed_conv_models()

precomputing conv. model outputs..
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
('models saved to files: ', 'precomputed_trn_features.segmented.h5', ' and ', 'precomputed_val_features.segmented.h5')
done.
('models saved to files: ', 'precomputed_trn_features.segmented.h5', ' and ', 'precomputed_val_features.segmented.h5')


<Executor.Executor instance at 0x7fe3866fe368>

In [3]:
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7feec6931368>

In [5]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.5)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)


In [6]:
for layer in fc_model.layers:
    layer.trainable = False

fc_model.layers[-1].trainable = True

In [7]:
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 2s - loss: 1.6952 - acc: 0.4429 - val_loss: 1.0177 - val_acc: 0.5448
Epoch 2/5
6166/6166 [==============================] - 2s - loss: 1.3942 - acc: 0.4737 - val_loss: 0.9515 - val_acc: 0.5517
Epoch 3/5
6166/6166 [==============================] - 2s - loss: 1.2555 - acc: 0.5021 - val_loss: 0.9492 - val_acc: 0.5380
Epoch 4/5
6166/6166 [==============================] - 2s - loss: 1.1718 - acc: 0.5101 - val_loss: 0.9406 - val_acc: 0.5544
Epoch 5/5
6166/6166 [==============================] - 2s - loss: 1.1013 - acc: 0.5178 - val_loss: 0.9002 - val_acc: 0.5654


In [8]:
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=10,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/10
6166/6166 [==============================] - 2s - loss: 1.0095 - acc: 0.5574 - val_loss: 0.8793 - val_acc: 0.5702
Epoch 2/10
6166/6166 [==============================] - 2s - loss: 0.9954 - acc: 0.5509 - val_loss: 0.8769 - val_acc: 0.5688
Epoch 3/10
6166/6166 [==============================] - 2s - loss: 0.9883 - acc: 0.5474 - val_loss: 0.8800 - val_acc: 0.5640
Epoch 4/10
6166/6166 [==============================] - 2s - loss: 0.9812 - acc: 0.5555 - val_loss: 0.8716 - val_acc: 0.5729
Epoch 5/10
6166/6166 [==============================] - 2s - loss: 0.9865 - acc: 0.5482 - val_loss: 0.8731 - val_acc: 0.5681
Epoch 6/10
6166/6166 [==============================] - 2s - loss: 0.9623 - acc: 0.5647 - val_loss: 0.8746 - val_acc: 0.5715
Epoch 7/10
6166/6166 [==============================] - 2s - loss: 0.9458 - acc: 0.5631 - val_loss: 0.8695 - val_acc: 0.5654
Epoch 8/10
6166/6166 [==================

In [9]:
for layer in fc_model.layers:
    layer.trainable = True

In [10]:
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=3,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/3
6166/6166 [==============================] - 7s - loss: 0.9122 - acc: 0.5733 - val_loss: 0.8373 - val_acc: 0.6003
Epoch 2/3
6166/6166 [==============================] - 7s - loss: 0.7778 - acc: 0.6505 - val_loss: 0.8138 - val_acc: 0.6044
Epoch 3/3
6166/6166 [==============================] - 7s - loss: 0.6682 - acc: 0.7052 - val_loss: 0.7892 - val_acc: 0.6201


In [11]:
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=3,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/3
6166/6166 [==============================] - 7s - loss: 0.5738 - acc: 0.7611 - val_loss: 0.7631 - val_acc: 0.6434
Epoch 2/3
6166/6166 [==============================] - 7s - loss: 0.4801 - acc: 0.8064 - val_loss: 0.7475 - val_acc: 0.6468
Epoch 3/3
6166/6166 [==============================] - 7s - loss: 0.3978 - acc: 0.8495 - val_loss: 0.7324 - val_acc: 0.6728


In [12]:
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=2,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/2
6166/6166 [==============================] - 7s - loss: 0.3169 - acc: 0.8883 - val_loss: 0.7301 - val_acc: 0.6783
Epoch 2/2
6166/6166 [==============================] - 7s - loss: 0.2538 - acc: 0.9137 - val_loss: 0.7537 - val_acc: 0.6687


In [13]:
fc_model.save_weights("weights_linear_model.segmented.all_layers_trained.h5")

In [14]:
opt = Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 7s - loss: 0.2005 - acc: 0.9382 - val_loss: 0.7379 - val_acc: 0.6838
Epoch 2/5
6166/6166 [==============================] - 7s - loss: 0.1840 - acc: 0.9489 - val_loss: 0.7394 - val_acc: 0.6845
Epoch 3/5
6166/6166 [==============================] - 7s - loss: 0.1816 - acc: 0.9475 - val_loss: 0.7383 - val_acc: 0.6879
Epoch 4/5
6166/6166 [==============================] - 7s - loss: 0.1768 - acc: 0.9492 - val_loss: 0.7414 - val_acc: 0.6845
Epoch 5/5
6166/6166 [==============================] - 7s - loss: 0.1696 - acc: 0.9515 - val_loss: 0.7430 - val_acc: 0.6838


In [15]:
executor.init_vgg_with_retrained_fc_layers(fc_model)

<Executor.Executor instance at 0x7fe3866fe368>

In [16]:
executor.save_model_to_file()

<Executor.Executor instance at 0x7fe3866fe368>

In [17]:
executor.build_predictions_on_test_data()

Found 512 images belonging to 1 classes.


TypeError: sequence item 0: expected string, numpy.ndarray found

In [20]:
outF = open("predictions.segmented.h5", 'w')
outF.write('image_name,Type_1,Type_2,Type_3\n')

for elem in executor.prediction:
    outF.write(elem[0] + ',' + ','.join(str(elem[1])) + '\n')
outF.close()

In [22]:
from IPython.display import FileLink
# bad content
FileLink("predictions.segmented.h5")

/home/ubuntu/kaggle_cervical_cancer_code/src/predictions.segmented.h5

In [23]:
for elem in executor.prediction:
   print elem
   print '====================='
outF.close()

('400.jpg', array([[['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ..., 
        ['0.0', '0.0', '0.0', ..., '20.2313957214', '1.40481328964', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0']],

       [['0.0', '0.0', '0.0', ..., '13.8067121506', '4.9282002449', '0.0'],
        ['0.0', '0.0', '0.0', ..., '9.45267105103', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '12.9296684265', '0.0', '0.0'],
        ..., 
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0']],

       [['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
        ['3.43382835388', '0.0', '2.53236865997', ..., '0.0', '0.0', '0.0'],
        ['0.0', '0.903492689133', '7.02557277679', ..., '0.0', '0.0', '0.0'],
 